In [ ]:
"""
DS 5110 Assignment 4: Part 1: GPT2 Pretraining (using the Transformers library)

This Notebook file contains a GPT2 pretraining workflow using the HuggingFace Transformers library.
You will implement the training loop to understand the GPT training process.
The skeleton code of helper functions is provided.
"""

'''
DO NOT MODIFY THIS CELL.
This cell contains core imports and configuration parameters for the training process.
'''
import os
import time
import math
import pickle
from contextlib import nullcontext
import inspect
import requests

import numpy as np
import torch
from torch.optim import AdamW
from torch.nn import functional as F
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.distributed import init_process_group, destroy_process_group

# Import transformers library
from transformers import (
    GPT2Config, 
    GPT2LMHeadModel, 
    GPT2Tokenizer,
    get_linear_schedule_with_warmup
)

In [ ]:
'''
This section prepares the dataset for character-level language modeling.
However, we'll use the transformers tokenizer for the model instead of character mapping.
'''
# Download and prepare the dataset
input_file_path = os.path.join(os.getcwd(), 'input.txt')
gpt_text = """
Generative Pre-trained Transformer (GPT) is a family of large-scale artificial intelligence models developed by OpenAI. 
Built on the transformer architecture, GPT is designed to process and generate human-like text by learning from vast datasets. 
Since its initial release in 2018, the GPT model family has evolved through several iterations, 
each demonstrating significant improvements in scale, performance, and capabilities."""

# Write the custom text to the file
with open(input_file_path, 'w') as f:
    f.write(gpt_text)

with open(input_file_path, 'r') as f:
    data = f.read()
print(f"length of dataset in characters: {len(data):,}")

# -----------------------------------------------------------------------------
# Default configuration values
# -----------------------------------------------------------------------------
# I/O
out_dir = 'out'
eval_interval = 1000  # Checkpoint interval
log_interval = 1      # Training loss printing interval
eval_iters = 100      # Estimate loss on train and validation splits by averaging over eval_iters batches
eval_only = False 
always_save_checkpoint = True
init_from = 'scratch' # 'scratch' or 'resume' or 'gpt2*'
# data
dataset = ''
gradient_accumulation_steps = 1 
batch_size = 12 
block_size = 64  # Context window size
# model
n_layer = 4       # Number of transformer layers
n_head = 4        # Number of attention heads
n_embd = 128      # Embedding dimension
dropout = 0.0     # Dropout probability
bias = False      # Whether to use bias in LayerNorm and Linear layers
# optimizer
learning_rate = 6e-5
max_iters = 2000  # Maximum number of training iterations
weight_decay = 0.01
beta1 = 0.9
beta2 = 0.95
# learning rate decay settings
decay_lr = True
warmup_iters = 0  
lr_decay_iters = 1500
min_lr = 6e-6
# system
device = 'cuda' if torch.cuda.is_available() else 'cpu' # Training uses CPU by default 
dtype = 'float32'  # Simplified to avoid mixed precision
device_type = 'cuda' if 'cuda' in device else 'cpu'
ctx = nullcontext()  # No mixed precision context
pad_token_id = 50256  # GPT2 EOS token
# -----------------------------------------------------------------------------

In [ ]:
'''
DO NOT MODIFY THIS CELL.
This function sets up the training environment, including directories and random seed.
It ensures consistency between runs and enables proper hardware utilization.
'''
def setup_training_environment():
    """Setup training environment, directories, and random seed."""
    os.makedirs(out_dir, exist_ok=True)
    torch.manual_seed(1337)
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True
    print(f"Running on device: {device}")
    tokens_per_iter = gradient_accumulation_steps * batch_size * block_size
    print(f"Tokens per iteration: {tokens_per_iter:,}")

In [ ]:
'''
Model initialization function using the Transformers library.
This replaces the custom model implementation with HuggingFace's GPT2LMHeadModel.
'''
def initialize_model_and_tokenizer():
    """Initialize a GPT model and tokenizer using the transformers library."""
    print("Initializing model using Transformers library")
    
    # Initialize tokenizer from pretrained GPT2
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    
    # Create model configuration with appropriate parameters
    config = GPT2Config(
        vocab_size=tokenizer.vocab_size,
        n_positions=block_size,
        n_ctx=block_size,
        n_embd=n_embd,
        n_layer=n_layer,
        n_head=n_head,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
    )
    
    # Initialize the model with the configuration
    model = GPT2LMHeadModel(config)
    
    # Move model to device
    model.to(device)
    
    # Print model parameter count
    num_params = sum(p.numel() for p in model.parameters())
    print(f"Number of parameters: {num_params:,}")
    
    # Store model arguments for checkpoint saving
    model_args = {
        'n_layer': n_layer,
        'n_head': n_head,
        'n_embd': n_embd,
        'block_size': block_size, 
        'dropout': dropout,
        'vocab_size': tokenizer.vocab_size
    }
    
    return model, tokenizer, model_args

In [ ]:
'''
Preprocess and tokenize the input data using the Transformers tokenizer.
'''
def process_dataset(tokenizer):
    """Process and tokenize the dataset using the Transformers tokenizer."""
    print("Processing dataset with Transformers tokenizer")
    
    # Read the input file
    with open(input_file_path, 'r', encoding='utf-8') as f:
        data = f.read()
    
    # Tokenize the data
    # Add special tokens to ensure we have enough content
    # Repeat the data a few times to make the dataset larger
    data_repeated = data * 3  # Repeat the text 3 times to make it larger
    encoded_data = tokenizer.encode(data_repeated)
    
    # Create the train and test splits
    n = len(encoded_data)
    train_data = encoded_data[:int(n*0.9)]
    val_data = encoded_data[int(n*0.9):]
    
    print(f"train has {len(train_data):,} tokens")
    print(f"val has {len(val_data):,} tokens")
    
    # Export to bin files
    train_ids = np.array(train_data, dtype=np.uint16)
    val_ids = np.array(val_data, dtype=np.uint16)
    train_ids.tofile(os.path.join(os.getcwd(), 'train.bin'))
    val_ids.tofile(os.path.join(os.getcwd(), 'val.bin'))
    
    return train_ids, val_ids

In [ ]:
'''
Get a batch of data from either the training or validation split.
This function is updated to handle cases where the data is smaller than block_size.
'''
def get_batch(split, step):
    """
    Get a batch of data from either 'train' or 'val' split.
    `step` is the global training step (or batch number).
    """
    # Load data
    if split == 'train':
        data = np.memmap(os.path.join(os.getcwd(), 'train.bin'), dtype=np.uint16, mode='r')
    else:
        data = np.memmap(os.path.join(os.getcwd(), 'val.bin'), dtype=np.uint16, mode='r')
    
    data_size = len(data)
    
    # Create empty tensors with padding
    x = torch.full((batch_size, block_size), pad_token_id, dtype=torch.long)
    y = torch.full((batch_size, block_size), pad_token_id, dtype=torch.long)
    
    # Calculate how many tokens we need for each batch element
    tokens_per_batch = batch_size * block_size
    
    for i in range(batch_size):
        # Calculate starting indices, wrapping around the dataset if needed
        idx = ((step * batch_size + i) * block_size) % (data_size - 1)
        
        # Determine how many tokens we can actually use (prevent overflow)
        available_tokens = min(block_size, data_size - 1 - idx)
        
        if available_tokens <= 0:
            # Skip this batch element if we're at the end of data
            continue
            
        # Get input sequence (x) 
        curr_x = torch.tensor(data[idx:idx+available_tokens], dtype=torch.long)
        # Get target sequence (y) - shifted by 1
        curr_y = torch.tensor(data[idx+1:idx+1+available_tokens], dtype=torch.long)
        
        # Place the sequences in our batch tensors
        x[i, :available_tokens] = curr_x
        y[i, :available_tokens] = curr_y
    
    # Move to device
    if device_type == 'cuda':
        x, y = x.pin_memory().to(device, non_blocking=True), y.pin_memory().to(device, non_blocking=True)
    else:
        x, y = x.to(device), y.to(device)
    
    return x, y

In [ ]:
'''
Learning rate scheduler compatible with the Transformers library.
'''
def get_lr(it):
    """Get learning rate based on current iteration."""
    # 1) linear warmup for warmup_iters steps
    if it < warmup_iters:
        return learning_rate * (it + 1) / (warmup_iters + 1)
    # 2) if it > lr_decay_iters, return min learning rate
    if it > lr_decay_iters:
        return min_lr
    # 3) in between, use cosine decay down to min learning rate
    decay_ratio = (it - warmup_iters) / (lr_decay_iters - warmup_iters)
    assert 0 <= decay_ratio <= 1
    coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio)) # coeff ranges 0..1
    return min_lr + coeff * (learning_rate - min_lr)

In [ ]:
'''
Estimate the model's loss on training and validation data.
Adapted to work with the Transformers model.
'''
@torch.no_grad()
def estimate_loss(model):
    """
    Estimate loss on train and validation splits by averaging over eval_iters batches.
    Returns a dictionary with keys 'train' and 'val', containing average losses.
    """
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split, k)
            outputs = model(X, labels=Y)
            losses[k] = outputs.loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [ ]:
'''
Set up the optimizer and learning rate scheduler using Transformers utilities.
'''
def setup_optimizer(model):
    """Configure the AdamW optimizer and scheduler for training."""
    # Group parameters for weight decay
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": weight_decay,
        },
        {
            "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
            "weight_decay": 0.0,
        },
    ]
    
    # Initialize optimizer
    optimizer = AdamW(
        optimizer_grouped_parameters,
        lr=learning_rate,
        betas=(beta1, beta2)
    )
    
    # Initialize scheduler
    if decay_lr:
        scheduler = get_linear_schedule_with_warmup(
            optimizer, 
            num_warmup_steps=warmup_iters, 
            num_training_steps=max_iters
        )
    else:
        scheduler = None
    
    return optimizer, scheduler

In [ ]:
'''
Training step function for forward and backward passes.
'''
def training_step(model, X, Y, gradient_accumulation_steps):
    """Implement forward and backward passes with Transformers model."""
    # Forward pass with transformers model
    outputs = model(X, labels=Y)
    loss = outputs.loss
    
    # Scale loss for gradient accumulation
    loss = loss / gradient_accumulation_steps
    
    # Perform backward pass
    loss.backward()
    
    return loss.item() * gradient_accumulation_steps  # Return unscaled loss for logging

In [ ]:
'''
Optimization step function to update model parameters.
'''
def optimization_step(model, optimizer, scheduler=None):
    """Update model parameters with optimizer."""
    # Gradient clipping (recommended for transformers)
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    
    # Step optimizer
    optimizer.step()
    
    # Step scheduler if provided
    if scheduler is not None:
        scheduler.step()
    
    # Zero gradients
    optimizer.zero_grad(set_to_none=True)

In [ ]:
'''
Save model checkpoints in the Transformers format for later fine-tuning.
'''
def save_checkpoint(model, tokenizer, optimizer, scheduler, iter_num, best_val_loss, model_args, is_final=False):
    t0 = time.time()
    
    if is_final:
        print(f"Saving final model using Hugging Face Transformers format to {out_dir}")
        save_dir = os.path.join(out_dir, "final_model")  # Where the final trained model will be saved
        
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        
        # Save model using Hugging Face format
        model.save_pretrained(save_dir)
        
        # Save tokenizer
        tokenizer.save_pretrained(save_dir)
    
    # Save additional information in a separate checkpoint
    checkpoint = {
        'model_args': model_args,
        'iter_num': iter_num,
        'best_val_loss': best_val_loss,
        'optimizer_state': optimizer.state_dict(),
        'scheduler_state': scheduler.state_dict() if scheduler is not None else None,
    }
    
    checkpoint_path = os.path.join(out_dir, 'ckpt.pt')
    print(f"Saving checkpoint to {checkpoint_path}")
    torch.save(checkpoint, checkpoint_path)  # Save ckpt.pt to the directory of checkpoint_path
    t1 = time.time()
    df = t1 - t0
    print(f"Post saving checkpoint: checkpointing took {dt:.2f}s")

In [ ]:
'''
Generate text using the trained model with Transformers.
'''
@torch.no_grad()
def generate_sample(model, tokenizer, max_new_tokens=50, temperature=0.1, top_k=10):
    """
    Generate a sample text from the trained model using Transformers.
    
    Args:
        model: The trained GPT model
        tokenizer: The tokenizer to use
        max_new_tokens: Maximum number of tokens to generate
        temperature: Controls randomness (lower = more deterministic)
        top_k: Controls diversity by limiting to top k options
    
    Returns:
        The generated text as a string
    """
    model.eval()
    
    # Start with "GPT" as context
    context = tokenizer.encode("GPT", return_tensors="pt").to(device)
    
    # Use the model's generate method with specified parameters
    output = model.generate(
        context, 
        max_length=max_new_tokens + len(context[0]),
        temperature=temperature,
        top_k=top_k,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        num_return_sequences=1
    )
    
    # Decode the generated tokens
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    
    return generated_text

In [ ]:
'''
Run the entire training process.
'''

"""
TODO: Task 1
"""
# Hint: Setup training environment
# Hint: Initialize model
# Hint: Setup optimizer
# Hint: Setup training variables (w/ setup_training())
# Hint: Get first batch ready (get_batch())

# TODO: Your code here...

"""
End of Task 1
"""

# Initialize training variables
iter_num = 0
best_validation_loss = float('inf') # Keep track of the best validation loss (lower the better)


# Start the timer
t0 = time.time()
total_time = 0

print("Beginning training...")

# Set model to training mode
model.train()

while iter_num < max_iters:
    # Fill the blanks below to implement a single training iteration
    
    # Update learning rate manually if not using scheduler and decay_lr is True
    if scheduler is None and decay_lr:
        """
        TODO: Task 2
        """
        # Hint: Get the learning rate by callined get_lr()
        # Hint: Update learning rate manually for each param_group in optimizer.param_groups

        # Your code here... 
        
        """
        End of Task 2
        """
        
    # Evaluate and save model checkpoints at regular intervals
    if iter_num % eval_interval == 0:
        """
        TODO: Task 3
        """
        # Hint: Evaluate model by calling estimate_loss()
        # Hint: Record best_validation_loss
        # Hint: Save checkpoint by calling save_checkpoint()
        # Hint: Don't save checkpoint right at the start (when iter_num == 0)
                
        # Your code here...

        """
        End of Task 3
        """
    
    # Inner loop with gradient accumulation
    loss_value = 0
    for micro_step in range(gradient_accumulation_steps):
        """
        TODO: Task 4
        """
        # Hint: Perform forward and backward pass by calling training_step()
        # Hint: Average the loss across gradient_accumulation_steps by
        # Hint: loss_value += loss / gradient_accumulation_steps
        # Hint: Prefetch next batch of input (X) and target (Y)
                
        # Your code here...

        """
        End of Task 4
        """
        
    
    """
    TODO: Task 5
    """
    # Hint: Perform optimization by calling (optimization_step)
    # Your code here... 
    
    """
    End of Task 5
    """

    """
    DO NOT MODIFY this code block
    """
    # Timing and logging
    t1 = time.time()
    dt = t1 - t0
    t0 = t1
    total_time += dt

    # Print training log periodically
    if iter_num % log_interval == 0:
        # Get current learning rate
        current_lr = scheduler.get_last_lr()[0] if scheduler is not None else optimizer.param_groups[0]['lr']
            
        # Simple logging for progress tracking
        print(f"iter {iter_num}: loss {loss_value:.4f}, lr {current_lr:.2e}, iter_time {dt:.2f}s, total_time: {total_time:.2f}s")
            
    # Increment iteration counter
    iter_num += 1
    """
    End of DO NOT MODIFY the above code block
    """
    # End of the training loop


print("Training complete!")

# Generate sample text from the trained model
sample_text = generate_sample(model, tokenizer)
print("\nSample generated text:") 
print(sample_text)

# Save the final model in Hugging Face format
print("Saving the final model in HuggingFace Transformers format...")
save_checkpoint(model, tokenizer, optimizer, scheduler, iter_num, best_val_loss, model_args, is_final=True)
print(f"Model saved to {out_dir}/final_model")